**my version**:

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/coursera_course_dataset_v2_no_null.csv')

In [ ]:
class CourseRecommender:
    def __init__(self, data, title_weight=0.3, skills_weight=0.7):
        """
        Initialize the recommender system with a DataFrame containing 'Title' and 'Skills' columns

        Parameters:
        data: DataFrame with 'Title' and 'Skills' columns
        title_weight: Weight given to title similarity (default: 0.3)
        skills_weight: Weight given to skills similarity (default: 0.7)
        """
        self.df = data
        self.title_weight = title_weight
        self.skills_weight = skills_weight

        # Initialize TF-IDF vectorizers for both title and skills
        self.title_tfidf = TfidfVectorizer(stop_words='english')
        self.skills_tfidf = TfidfVectorizer(stop_words='english')

        # Fill NaN values
        self.df['Title'] = self.df['Title'].fillna('')
        self.df['Skills'] = self.df['Skills'].fillna('')

        # Create TF-IDF matrices for both title and skills
        self.title_matrix = self.title_tfidf.fit_transform(self.df['Title'])
        self.skills_matrix = self.skills_tfidf.fit_transform(self.df['Skills'])
        # print(self.skills_matrix.shape)
        # print(self.title_matrix.shape)
        # Calculate combined similarity matrix
        self._calculate_combined_similarity()

    def _calculate_combined_similarity(self):
        """
        Calculate weighted combination of title and skills similarity matrices
        """
        # Calculate separate similarity matrices
        title_sim = cosine_similarity(self.title_matrix, self.title_matrix)
        skills_sim = cosine_similarity(self.skills_matrix, self.skills_matrix)

        # Combine similarities with weights
        self.cosine_sim = (
            self.title_weight * title_sim +
            self.skills_weight * skills_sim
        )

    def get_recommendations(self, query_title, query_skills, num_recommendations=5):
        """
        Get course recommendations based on input title and skills

        Parameters:
        query_title (str): Title to match against
        query_skills (str): Comma-separated string of skills
        num_recommendations (int): Number of recommendations to return

        Returns:
        list: List of recommended course titles and their similarity scores
        """
        # Transform query using both vectorizers
        query_title_vector = self.title_tfidf.transform([query_title])
        query_skills_vector = self.skills_tfidf.transform([query_skills])

        # Calculate separate similarities
        title_sim = cosine_similarity(query_title_vector, self.title_matrix).flatten()
        skills_sim = cosine_similarity(query_skills_vector, self.skills_matrix).flatten()

        # Combine similarities with weights
        combined_sim = (
            self.title_weight * title_sim +
            self.skills_weight * skills_sim
        )

        # Get indices of top similar courses
        course_indices = combined_sim.argsort()[::-1][:num_recommendations]

        # Create list of recommendations with similarity scores
        recommendations = []
        for idx in course_indices:
            if combined_sim[idx] > 0:  # Only include if there's some similarity
                recommendations.append({
                    'title': self.df.iloc[idx]['Title'],
                    'skills': self.df.iloc[idx]['Skills'],
                    'similarity_score': round(combined_sim[idx] * 100, 2),
                    'title_similarity': round(title_sim[idx] * 100, 2),
                    'skills_similarity': round(skills_sim[idx] * 100, 2)
                })

        return recommendations

# Example usage:
def main():

    # Initialize recommender with custom weights
    recommender = CourseRecommender(data, title_weight=0.3, skills_weight=0.7)

    # Example recommendation request
    query_title = "Machine Learning Course"
    query_skills = "python, data analysis, algorithms"
    recommendations = recommender.get_recommendations(query_title, query_skills)

    # Print recommendations
    print(f"\nRecommendations for:")
    print(f"Title: {query_title}")
    print(f"Skills: {query_skills}\n")

    for i, rec in enumerate(recommendations, 1):
        print(f"{i}. {rec['title']}")
        print(f"   Skills: {rec['skills']}")
        print(f"   Overall Similarity: {rec['similarity_score']}%")
        print(f"   Title Similarity: {rec['title_similarity']}%")
        print(f"   Skills Similarity: {rec['skills_similarity']}%\n")

if __name__ == "__main__":
    main()


Recommendations for:
Title: Machine Learning Course
Skills: python, data analysis, algorithms

1. Data Analysis Using Pyspark
   Skills:  Data Analysis, Python Programming
   Overall Similarity: 47.73%
   Title Similarity: 0.0%
   Skills Similarity: 68.18%

2. Exploratory Data Analysis With Python and Pandas
   Skills:  Data Analysis, Exploratory Data Analysis, Python Programming
   Overall Similarity: 43.05%
   Title Similarity: 0.0%
   Skills Similarity: 61.49%

3. Python for Data Analysis: Pandas & NumPy
   Skills:  Data Analysis, Python Programming, Data Science
   Overall Similarity: 41.49%
   Title Similarity: 0.0%
   Skills Similarity: 59.27%

4. ChatGPT Advanced Data Analysis
   Skills:  Data Analysis
   Overall Similarity: 39.31%
   Title Similarity: 0.0%
   Skills Similarity: 56.16%

5. Investment Management with Python and Machine Learning
   Skills:  Investment Management, Finance, Python Programming, Data Analysis, Computer Programming, Machine Learning, Statistical Progr